In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
from langchain_groq import ChatGroq

model = ChatGroq(model="openai/gpt-oss-120b", temperature=0)

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

system_prompt = """You are a travel agent. 
IMPORTANT: When using the search-flight tool, you MUST use the following rules:
1. Dates must be in 'dd/mm/yyyy' format (e.g., 31/03/2026).
2. All dates must be in the future.
3. If the user asks for a one-way flight, ensure you still provide the correct parameters required by the tool.
No follow up questions."""

agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt=system_prompt
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="""Create an effective prompt template for getting flights""")]},
    config
    )

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Create an effective prompt template for getting flights', additional_kwargs={}, response_metadata={}, id='6e681292-113f-4ce3-90a9-c3cc90112504'),
              AIMessage(content='Below is a ready‑to‑copy prompt you can paste into a chat with a travel‑assistant (or any AI that has access to the **search‑flight** tool).  \nJust replace the placeholders with your own details and hit “send”. The assistant will automatically query the Kiwi API and return a nicely formatted list of options.\n\n---\n\n## Flight‑search Prompt Template  \n\n```\n/search-flight\n{\n  "flyFrom": "<DEPARTURE CITY OR AIRPORT CODE>",          // e.g. “NYC”, “London LHR”, “Paris CDG”\n  "flyTo": "<DESTINATION CITY OR AIRPORT CODE>",         // e.g. “Tokyo NRT”, “Sydney SYD”\n  "departureDate": "<DD/MM/YYYY>",                       // must be a future date, format dd/mm/yyyy\n  "departureDateFlexRange": <0‑3>,                       // optional – days before/after you can be flexible

In [ ]:
print(response["messages"][-1].content)

In [ ]:
prompt = """I need to book flights for 2 adults and 1 child from New York to London. 
            We plan to depart on July 15, 2026, with some flexibility of ±1 day around 
            that date. Our return date is August 15, 2026, also with ±1 day flexibility. 
            Please search for economy class (M) flights and show me results sorted by 
            price from lowest to highest. Display the prices in USD."""


response = await agent.ainvoke(
    {"messages": [HumanMessage(content=prompt)]},
    config
    )


pprint(response)

In [ ]:
print(response["messages"][-1].content)